In [ ]:
# Imports
import GetOldTweets3 as got
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from unidecode import unidecode #retirar acentos
import string
string.punctuation
from wordcloud import WordCloud,ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import regex as re

In [ ]:
df_keywords = pd.read_csv('keywords_fakenews_covid.csv', sep=';', encoding='ISO-8859-1', error_bad_lines=False)
df_keywords.head(12)

In [ ]:
def search_tweets(queries, subject, top_only, start_date, end_date, max_tweets):
    
    tweets_df = pd.DataFrame(columns = ['user', 'text','date', 'favorites', 'retweets', 'mentions', 'hashTags','id', 'subject'])
    
    for i, query in enumerate(queries):
        tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                              .setTopTweets(top_only)\
                              .setSince(start_date)\
                              .setUntil(end_date)\
                              .setMaxTweets(max_tweets)\
                              .setLang('pt')

        # Creation of list that contains all tweets
        tweet = got.manager.TweetManager.getTweets(tweetCriteria)

        # Creating list of chosen tweet data
        text_tweets = [[tw.username,
                    tw.text,
                    tw.date,
                    tw.retweets,
                    tw.favorites,
                    tw.mentions,
                    tw.hashtags,
                    tw.id] for tw in tweet]

        # creating dataframe, assigning column names to list of
        tweets = pd.DataFrame(text_tweets, 
                                columns = ['user', 'text','date', 'favorites', 'retweets', 'mentions', 'hashTags','id'])
        tweets['subject'] = subject[i]
        
        tweets_df = tweets_df.append(tweets)
            
    return tweets_df

Criando a query que será pesquisada pelo crawler:

In [ ]:
df_keywords.keywords

In [ ]:
search = df_keywords.keywords.apply(lambda x: x + ' AND (Covid OR Corona OR covid OR covid19 OR corona OR coronavírus OR coronavirus)')

search_query = pd.DataFrame({'keywords': search[5:7].values, 'subject': df_keywords['assunto_fake'][5:7].values})
search_query

In [ ]:
search_query.keywords

Capturando os tweets com as palavras chaves:

In [ ]:
pool_tweets = search_tweets(search_query.keywords, search_query.subject,
                                        top_only = True,
                                        start_date = "2020-02-01", 
                                        end_date = "2020-09-09",
                                        max_tweets = 100000)

In [ ]:
pool_tweets.subject.value_counts()#[200:300].text.tolist()

Apagando tweets duplicados:

In [ ]:
pool_tweets = pool_tweets.drop_duplicates(subset='id', keep='first')

Salvando em csv:

In [ ]:
pool_tweets.to_csv('tweets_Topics6-7.csv', header=True, encoding='utf -8-sig')